<h2>Objectifs</h2>

L'objectif de ce code est d'extrire les données météorologiqus d'intérêt (vent et précipitation en heure par heure à Paris). 
Les données proviennent de https://meteo.data.gouv.fr/datasets/6569b4473bedf2e7abad3b72

In [4]:
import pandas as pd

df = pd.read_csv('../dataset_brut/H_75_latest-2024-2025.csv', sep=";", encoding='latin-1')

df = df[(df['NUM_POSTE'] == 75114001)].reset_index(drop=True)

df = df.loc[:,['NOM_USUEL','AAAAMMJJHH','Pr\x8ecipitation en 1h','Dur\x8ee precipitation (en min)','Force du vent moyenne)']]

df = df.rename(columns={'Pr\x8ecipitation en 1h': 'precipitations heure', 
                        'Dur\x8ee precipitation (en min)' : 'duree prec (en min)',
                        'Force du vent moyenne)' : 'force moyenne vent (m/s)'})

df['datetime'] = pd.to_datetime(df['AAAAMMJJHH'].astype(str), format='%Y%m%d%H')

df = df.drop('AAAAMMJJHH', axis=1)

df.to_csv('../dataset/meteo.csv', sep=';')

<h2>Remplissage des données manquantes</h2>

Nous remplissons les données manquantes par la moyenne entre les deux dernières observations effectuées. 

In [7]:
print("Le dataset comporte " + str(df.shape[0]) + " lignes")
df.info()

Le dataset comporte 16036 lignes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16036 entries, 0 to 16035
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   NOM_USUEL                 16036 non-null  object        
 1   precipitations heure      16035 non-null  float64       
 2   duree prec (en min)       15935 non-null  float64       
 3   force moyenne vent (m/s)  16033 non-null  float64       
 4   datetime                  16036 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 626.5+ KB


Les données sont quasiment toutes présentes, ce qui valide la décision de remplir par la moyenne des observations autour de celles manquantes.

In [ ]:
colonnes = df.columns.to_list()

for col in colonnes :
    print(df[col].isnull().sum())
    if df[col].isnull().sum() == df.shape[0]